In [28]:
import requests
import json
import pandas as pd
import time
from dotenv import load_dotenv
import os
import re
from datetime import datetime
from decimal import Decimal, InvalidOperation
import base64
from datetime import datetime, timedelta
import pytz

pd.options.display.float_format = '{:,.14f}'.format

In [29]:
# Configurações
load_dotenv()
TOKEN_URL = 'https://api.ticket360.com.br/auth/oauth/access_token'
API_URL = 'https://api.ticket360.com.br'
EVENT_ID = '30642'
MAX_RETRIES = 3  #Max request until fail
TIMEOUT = 30  # Seconds
DATA_FILE = 'ticket_data.json'

#get token from retries and timout adjustaded
def get_access_token():
    for attempt in range(MAX_RETRIES):
        try:
            auth_string = f"{os.getenv('CLIENT_ID')}:{os.getenv('CLIENT_SECRET')}"
            auth_base64 = base64.b64encode(auth_string.encode()).decode()
            
            response = requests.post(
                TOKEN_URL,
                headers={
                    "Authorization": f"Basic {auth_base64}",
                    "Content-Type": "application/x-www-form-urlencoded"
                },
                data={"grant_type": "client_credentials"},
                timeout=TIMEOUT
            )
            response.raise_for_status()
            return response.json().get("access_token")
            
        except requests.exceptions.Timeout:
            print(f"Timeout (attempt {attempt + 1}/{MAX_RETRIES})")
            if attempt < MAX_RETRIES - 1:
                time.sleep(5)
                continue
            raise 
        except Exception as e:
            print(f"Erro: {type(e).__name__} - {str(e)}")
            return None

#get date from url_api with error handling
# Função fetch_report com paginação e filtros de data
def fetch_report(token, start_date=None, end_date=None):
    try:
        base_url = f"{API_URL}/sales/reports/consolidated/{EVENT_ID}?filter=status=paid&ticket.status=active"
        
        # Adicionar filtros de data se fornecidos
        if start_date:
            base_url += f"&startDate={start_date}"
        if end_date:
            base_url += f"&endDate={end_date}"
        
        offset = 0
        limit = 1000
        all_sales = []
        
        while True:
            url = f"{base_url}&limit={limit}&offset={offset}"
            response = requests.get(
                url,
                headers={"Authorization": f"Bearer {token}"},
                timeout=TIMEOUT
            )
            response.raise_for_status()
            data = response.json()
            sales = data.get('sales', [])
            all_sales.extend(sales)
            
            print(f"Offset {offset}: {len(sales)} registros")
            
            # Se a quantidade de registros for menor que o limite, chegamos ao fim
            if len(sales) < limit:
                break
                
            offset += limit
        
        return {'sales': all_sales}
    except requests.exceptions.Timeout:
        print("Timeout when fetching report")
    except Exception as e:
        print(f"Report error: {type(e).__name__} - {str(e)}")
    return None

# Função para formatar datas para o padrão ISO com timezone
def format_date(date_obj, end_of_day=False):
    tz = pytz.timezone('America/Sao_Paulo')
    if end_of_day:
        date_obj = date_obj.replace(hour=23, minute=59, second=59)
    else:
        date_obj = date_obj.replace(hour=0, minute=0, second=0)
    return date_obj.astimezone(tz).isoformat()

# Função para salvar dados em JSON
def save_data(df, filename):
    # Se o arquivo já existe, combinamos os dados
    if os.path.exists(filename):
        try:
            df_existente = pd.read_json(filename, lines=True)
            # Combinar datasets
            df_final = pd.concat([df_existente, df])
            # Remover duplicatas
            df_final = df_final.drop_duplicates(subset='ticket.id', keep='last')
        except:
            df_final = df
    else:
        df_final = df
    
    # Salvar em formato JSON
    df_final.to_json(filename, orient='records', lines=True)
    return df_final





In [30]:
# Fluxo principal
if __name__ == "__main__":
    # Configurar timezone
    tz = pytz.timezone('America/Sao_Paulo')
    hoje = datetime.now(tz)
    dois_dias_atras = hoje - timedelta(days=2)
    ontem = hoje - timedelta(days=1)
    
    # Obter token
    token = get_access_token()
    if not token:
        print("Falha ao obter token. Abortando.")
        exit(1)
    
    # Verificar arquivo existente
    df_existente = pd.DataFrame()
    data_mais_recente = None
    
    if os.path.exists(DATA_FILE):
        try:
            df_existente = pd.read_json(DATA_FILE, lines=True)
            # Converter coluna de datas
            df_existente['date'] = pd.to_datetime(df_existente['date'])
            # Encontrar data mais recente
            if not df_existente.empty:
                data_mais_recente = df_existente['date'].max().to_pydatetime()
                data_mais_recente = data_mais_recente.astimezone(tz)
                print(f"Data mais recente no arquivo: {data_mais_recente}")
        except Exception as e:
            print(f"Erro ao ler arquivo: {e}")
    
    # Primeira execução: coletar todo o histórico até 2 dias atrás
    if data_mais_recente is None:
        print("Primeira execução: coletando todo o histórico até 2 dias atrás")
        fim_historico = format_date(dois_dias_atras, end_of_day=True)
        report = fetch_report(token, end_date=fim_historico)
        
        if report and 'sales' in report:
            df_historico = pd.DataFrame(report['sales'])
            if not df_historico.empty:
                # Converter datas
                df_historico['date'] = pd.to_datetime(df_historico['date'])
                # Salvar dados históricos
                df_final = save_data(df_historico, DATA_FILE)
                print(f"Dados históricos salvos: {len(df_final)} registros")
            else:
                print("Nenhum dado histórico encontrado")
        else:
            print("Falha ao coletar dados históricos")
    else:
        # Execuções subsequentes: coletar dados desde a última data até ontem
        print("Execução subsequente: coletando dados atualizados")
        
        # Coletar dados desde a última data até ontem
        inicio_atualizacao = format_date(data_mais_recente + timedelta(seconds=1))
        fim_atualizacao = format_date(ontem, end_of_day=True)
        
        print(f"Coletando dados de {inicio_atualizacao} até {fim_atualizacao}")
        report = fetch_report(token, start_date=inicio_atualizacao, end_date=fim_atualizacao)
        
        if report and 'sales' in report:
            df_novos = pd.DataFrame(report['sales'])
            if not df_novos.empty:
                # Converter datas
                df_novos['date'] = pd.to_datetime(df_novos['date'])
                # Salvar novos dados
                df_final = save_data(df_novos, DATA_FILE)
                print(f"Dados atualizados salvos: {len(df_final)} registros")
            else:
                print("Nenhum dado novo encontrado no período")
        else:
            print("Falha ao coletar dados atualizados")
    
    print("Processo concluído")

Data mais recente no arquivo: 2025-08-12 14:36:35-03:00
Execução subsequente: coletando dados atualizados
Coletando dados de 2025-08-12T00:00:00-03:00 até 2025-08-14T23:59:59.281153-03:00
Offset 0: 1000 registros
Offset 1000: 1000 registros
Offset 2000: 1000 registros
Offset 3000: 1000 registros
Offset 4000: 1000 registros
Offset 5000: 1000 registros
Offset 6000: 1000 registros
Offset 7000: 1000 registros
Offset 8000: 1000 registros
Offset 9000: 1000 registros
Offset 10000: 1000 registros
Offset 11000: 1000 registros
Offset 12000: 1000 registros
Offset 13000: 1000 registros
Offset 14000: 1000 registros
Offset 15000: 1000 registros
Offset 16000: 1000 registros
Offset 17000: 1000 registros
Offset 18000: 1000 registros
Offset 19000: 1000 registros
Offset 20000: 1000 registros
Offset 21000: 1000 registros
Offset 22000: 1000 registros
Offset 23000: 1000 registros
Offset 24000: 1000 registros
Offset 25000: 1000 registros
Offset 26000: 1000 registros
Offset 27000: 1000 registros
Offset 28000:

<h1>Code below is used for validate JSON files

In [35]:
'''
offset_json = "ticket_data.json"
path_json = os.path.join(os.getcwd(), offset_json)

df = pd.read_json(path_json, lines=True)
print(df.sort_values(by='date', ascending=False).head())
'''

'\noffset_json = "ticket_data.json"\npath_json = os.path.join(os.getcwd(), offset_json)\n\ndf = pd.read_json(path_json, lines=True)\nprint(df.sort_values(by=\'date\', ascending=False).head())\n'